# load package and model

In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys
sys.path

from config_ai.models import RelationTokenClassify
from config_ai.evaluate import eval_text_classify
from config_ai.models.relation_classify.common import *
from config_ai.data_utils import *
from config_ai.backend import set_tf_config
from config_ai.models import load_model
from config_ai.experiments import get_model_config
from config_ai.utils import read_config
from snippets.utils import *

os.environ["CUDA_VISIBLE_DEVICES"] = "1"
set_tf_config()


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


['/nfs/pony/chenhao/workspace/ConfigAI/notebooks/relation_classify',
 '/nfs/pony/chenhao/workspace/python-snippets',
 '/nfs/pony/chenhao/workspace/ConfigAI',
 '/nfs/pony/chenhao/workspace/bert4keras',
 '/nfs/pony/chenhao/workspace/ConfigAI/notebooks/relation_classify',
 '/nfs/pony/chenhao/envs/config_ai/lib/python38.zip',
 '/nfs/pony/chenhao/envs/config_ai/lib/python3.8',
 '/nfs/pony/chenhao/envs/config_ai/lib/python3.8/lib-dynload',
 '',
 '/nfs/pony/chenhao/envs/config_ai/lib/python3.8/site-packages',
 '/nfs/pony/chenhao/envs/config_ai/lib/python3.8/site-packages/IPython/extensions',
 '/mnt/raid4/home/chenhao/.ipython']

2021-11-23 11:38:23 [INFO][backend.py:57]:setting tensorflow config...
2021-11-23 11:38:23.450843: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2021-11-23 11:38:23 [INFO][backend.py:61]:current devices:
2021-11-23 11:38:23.463275: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:06:00.0 name: NVIDIA TITAN X (Pascal) computeCapability: 6.1
coreClock: 1.531GHz coreCount: 28 deviceMemorySize: 11.91GiB deviceMemoryBandwidth: 447.48GiB/s
2021-11-23 11:38:23.463702: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
2021-11-23 11:38:23.465499: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10
2021-11-23 11:38:23.467007: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.10
2021-1

In [2]:
config_path = "/nfs/pony/chenhao/workspace/ConfigAI/examples/relation_classify/relation_token_classify.ini"

config = read_config(config_path)
config
model_config = get_model_config(config)
# model_config

model = RelationTokenClassify(config=model_config)
model.id2label
model.embedding_strategy

2021-11-23 11:38:23 [INFO][utils.py:56]:parsing config with path:/nfs/pony/chenhao/workspace/ConfigAI/examples/relation_classify/relation_token_classify.ini
2021-11-23 11:38:23 [INFO][utils.py:67]:loading base config...
2021-11-23 11:38:23 [INFO][utils.py:56]:parsing config with path:/nfs/pony/chenhao/workspace/ConfigAI/examples/base_config.ini


{'common_config': {'experiment_dir': '/nfs/pony/chenhao/experiment',
  'project_name': 'config_ai_example',
  'is_train': True,
  'is_test': True,
  'is_save': True,
  'save_args': {'format': 'h5', 'tf_serving_version': 1},
  'eval_phase_list': ['train', 'dev'],
  'output_phase_list': ['train', 'dev', 'test'],
  'is_overwrite_experiment': True,
  'default_random_seed': 10,
  'base_config': '/nfs/pony/chenhao/workspace/ConfigAI/examples/base_config.ini',
  'model_cls': 'RelationTokenClassifyModel',
  'model_name': 'relation_token_classify_example'},
 'tokenizer_config': {'tokenizer_name': 'bert_word_piece',
  'tokenizer_args': {'vocabs': '/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12/vocab.txt'}},
 'nn_model_config': {'pretrained_model_tag': 'bert',
  'pretrained_model_path': '/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12'},
 'compile_config': {'optimizer_name': 'adam',
  'optimizer_args': {'learning_rate': 3e-05}},
 'train_config': {'epochs': 2,
  'batch_size': 32,
  

2021-11-23 11:38:23 [INFO][core.py:47]:init model with config:
2021-11-23 11:38:23 [INFO][relation_token_classify.py:68]:initializing tokenizer
2021-11-23 11:38:23 [INFO][relation_token_classify.py:75]:replacing special tokens:['[S:城市]', '[O:城市]', '[S:国家]', '[O:国家]', '[/S]', '[/O]'] to vocabs


{0: '首都'}

<EmbeddingStrategy.ENTITY_START: 'ENTITY_START'>

# process data

In [3]:
train_data_path = config['data_config']['train_data_path']
train_data = model.jload_lines(train_data_path)
f"{train_data_path}: {len(train_data)} items"


eval_data_path = config['data_config']['eval_data_path']
eval_data = model.jload_lines(eval_data_path)
f"{eval_data_path}: {len(eval_data)} items"


test_data_path = config['data_config']['test_data_path']
test_data = model.jload_lines(test_data_path)
f"{test_data_path}: {len(test_data)} items"



train_data[0]

'/nfs/pony/chenhao/workspace/ConfigAI/data/relation_classify/labeled.jsonl: 2 items'

'/nfs/pony/chenhao/workspace/ConfigAI/data/relation_classify/labeled.jsonl: 2 items'

'/nfs/pony/chenhao/workspace/ConfigAI/data/relation_classify/labeled.jsonl: 2 items'

LabeledRelationClassifyExample(text='北京是中国的首都', text_span1=TextSpan(text='北京', span=(0, 2), label='城市', prob=1.0), text_span2=TextSpan(text='中国', span=(3, 5), label='国家', prob=1.0), label=[Label(name='首都', prob=1.0)])

In [4]:
data_manager = DataManager.get_instance(model=model, data=train_data)

In [5]:
features = data_manager.get_features()
feature = next(features)
feature

{'full_text': '[S:城市]北京[/S]是[O:国家]中国[/O]的首都',
 'text': '[S:城市]北京[/S]是[O:国家]中国[/O]的首都',
 'extra_text': None,
 'token_ids': [101,
  10,
  1266,
  776,
  14,
  3221,
  13,
  704,
  1744,
  15,
  4638,
  7674,
  6963,
  102],
 'segment_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'tokens': ['[CLS]',
  '[S:城市]',
  '北',
  '京',
  '[/S]',
  '是',
  '[O:国家]',
  '中',
  '国',
  '[/O]',
  '的',
  '首',
  '都',
  '[SEP]'],
 'span_idxs': [1, 6, 4, 9],
 'labels': ['首都']}

In [6]:
records = data_manager.get_records(mode="train")
next(records)



{'idx': 0,
 'full_text': '[S:城市]北京[/S]是[O:国家]中国[/O]的首都',
 'text': '[S:城市]北京[/S]是[O:国家]中国[/O]的首都',
 'extra_text': None,
 'token_ids': [101,
  10,
  1266,
  776,
  14,
  3221,
  13,
  704,
  1744,
  15,
  4638,
  7674,
  6963,
  102],
 'segment_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'tokens': ['[CLS]',
  '[S:城市]',
  '北',
  '京',
  '[/S]',
  '是',
  '[O:国家]',
  '中',
  '国',
  '[/O]',
  '的',
  '首',
  '都',
  '[SEP]'],
 'span_idxs': [1, 6, 4, 9],
 'labels': ['首都'],
 'classify_output': [1]}

# build&compile model

In [7]:
from config_ai.models.relation_classify.relation_token_classify import EmbeddingStrategy
model.embedding_strategy
model.embedding_strategy == EmbeddingStrategy.ENTITY_START

<EmbeddingStrategy.ENTITY_START: 'ENTITY_START'>

True

In [11]:
nn_model_args = dict(**config["nn_model_config"])
# nn_model_args.update(transformer_kwargs=dict(dropout_rate=0.3))
nn_model_args

model.build_model(**nn_model_args)



{'pretrained_model_tag': 'bert',
 'pretrained_model_path': '/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12'}

2021-11-23 11:38:50 [INFO][tf_core.py:126]:number of devices: 1, use SINGLE scope
2021-11-23 11:38:50 [INFO][nn_models.py:56]:loading from pretrained weights: /nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12/model.ckpt
2021-11-23 11:38:52 [INFO][relation_token_classify.py:112]:nn model's summary:
2021-11-23 11:38:52 [INFO][layer_utils.py:192]:Model: "model_5"
2021-11-23 11:38:52 [INFO][layer_utils.py:193]:__________________________________________________________________________________________________
2021-11-23 11:38:52 [INFO][layer_utils.py:190]:Layer (type)                    Output Shape         Param #     Connected to                     
2021-11-23 11:38:52 [INFO][layer_utils.py:195]:==================================================================================================
2021-11-23 11:38:52 [INFO][layer_utils.py:190]:token_ids (InputLayer)          [(None, None)]       0                                            
2021-11-23 11:38:52 [INFO][layer_utils.py:259]:_

In [14]:
model.multi_label

True

In [15]:
compile_args = dict(**config["compile_config"])
# compile_args.update(rdrop_alpha=4)
compile_args
model.compile_model(**compile_args)


{'optimizer_name': 'adam', 'optimizer_args': {'learning_rate': 3e-05}}

2021-11-23 14:33:59 [INFO][relation_token_classify.py:118]:compiling model...
2021-11-23 14:33:59 [INFO][tf_core.py:126]:number of devices: 1, use SINGLE scope
2021-11-23 14:33:59 [INFO][losses.py:101]:build loss layer with loss function:<function binary_crossentropy at 0x7fbe3384bca0>
2021-11-23 14:33:59 [INFO][relation_token_classify.py:137]:Tensor("classify_output_3:0", shape=(None, 1), dtype=float32)
2021-11-23 14:33:59 [INFO][relation_token_classify.py:138]:Tensor("pos_weight_layer_2/Identity:0", shape=(None, 1), dtype=float32)
2021-11-23 14:33:59 [INFO][relation_token_classify.py:144]:training model's summary:
2021-11-23 14:33:59 [INFO][layer_utils.py:192]:Model: "train_model"
2021-11-23 14:33:59 [INFO][layer_utils.py:193]:__________________________________________________________________________________________________
2021-11-23 14:33:59 [INFO][layer_utils.py:190]:Layer (type)                    Output Shape         Param #     Connected to                     
2021-11-23 14:33

In [16]:
model.train_model.inputs
model.train_model.outputs
model.train_model.losses
model.train_model.metrics


[<tf.Tensor 'token_ids_2:0' shape=(None, None) dtype=int32>,
 <tf.Tensor 'segment_ids_2:0' shape=(None, None) dtype=int32>,
 <tf.Tensor 'span_idxs_2:0' shape=(None, 4) dtype=int32>,
 <tf.Tensor 'classify_output_3:0' shape=(None, 1) dtype=float32>]

[<tf.Tensor 'pos_weight_layer_2/Identity:0' shape=(None, 1) dtype=float32>]

[<tf.Tensor 'loss_layer_3/Identity:0' shape=(None,) dtype=float32>]

In [17]:
model.get_dataset_info("train")
model.get_dataset_info("test")

({'token_ids': 'int32',
  'segment_ids': 'int32',
  'span_idxs': 'int32',
  'classify_output': 'float32'},
 {'token_ids': (None,),
  'segment_ids': (None,),
  'span_idxs': (4,),
  'classify_output': (1,)})

({'token_ids': 'int32', 'segment_ids': 'int32', 'span_idxs': 'int32'},
 {'token_ids': (None,), 'segment_ids': (None,), 'span_idxs': (4,)})

In [18]:
dataset = data_manager.get_train_dataset(batch_size=1)

item = next(dataset.as_numpy_iterator())
item


{'token_ids': array([[ 101,   10, 1266,  776,   14, 3221,   13,  704, 1744,   15, 4638,
         7674, 6963,  102]], dtype=int32),
 'segment_ids': array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32),
 'span_idxs': array([[1, 6, 4, 9]], dtype=int32),
 'classify_output': array([[1.]], dtype=float32)}

# train_model

In [19]:
train_args = dict(**config["train_config"])
train_args.update(steps_per_epoch=100, batch_size=8, verbose=1)
train_args

model.train(train_data=train_data, **train_args)

{'epochs': 2,
 'batch_size': 8,
 'overwrite_cache': False,
 'steps_per_epoch': 100,
 'verbose': 1}

100%|██████████| 2/2 [00:00<00:00, 2238.75it/s]
2021-11-23 14:34:12 [INFO][tf_core.py:173]:train on 2 tensors


Epoch 1/2


2021-11-23 14:34:17.331607: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10


100/100 [==============================] - 12s 121ms/step - loss: 0.0211 - accuracy: 0.9937
Epoch 2/2
100/100 [==============================] - 12s 116ms/step - loss: 1.1912e-05 - accuracy: 1.0000


2021-11-23 14:34:42 [INFO][tf_core.py:189]:training finished


# predict&evaluate

In [20]:
to_pred = train_data[:4]
to_pred

pred = model.predict(to_pred, show_detail=True)
pred[:4]


[LabeledRelationClassifyExample(text='北京是中国的首都', text_span1=TextSpan(text='北京', span=(0, 2), label='城市', prob=1.0), text_span2=TextSpan(text='中国', span=(3, 5), label='国家', prob=1.0), label=[Label(name='首都', prob=1.0)]),
 LabeledRelationClassifyExample(text='上海不是中国的首都', text_span1=TextSpan(text='上海', span=(0, 2), label='城市', prob=1.0), text_span2=TextSpan(text='中国', span=(4, 6), label='国家', prob=1.0), label=[])]

100%|██████████| 2/2 [00:00<00:00, 2105.57it/s]
2021-11-23 14:35:18 [INFO][tf_core.py:204]:predicting with tf model...
1it [00:00,  6.75it/s]
2021-11-23 14:35:18 [INFO][decorators.py:28]:function:_model_predict cost:0.151 seconds
2021-11-23 14:35:18 [INFO][relation_token_classify.py:186]:pred tensor
2021-11-23 14:35:18 [INFO][relation_token_classify.py:187]:tf.Tensor([0.9999976], shape=(1,), dtype=float32)
2021-11-23 14:35:18 [INFO][relation_token_classify.py:186]:pred tensor
2021-11-23 14:35:18 [INFO][relation_token_classify.py:187]:tf.Tensor([1.8487617e-06], shape=(1,), dtype=float32)
2021-11-23 14:35:18 [INFO][decorators.py:28]:function:_post_predict cost:0.015 seconds


[[Label(name='首都', prob=0.999997615814209)], []]

In [25]:
output_data = get_relation_classify_output(to_pred, pred)
output_data[:4]

[{'text': '北京是中国的首都',
  'text_span1': {'text': '北京', 'span': (0, 2), 'label': '城市', 'prob': 1.0},
  'text_span2': {'text': '中国', 'span': (3, 5), 'label': '国家', 'prob': 1.0},
  'label': [{'name': '首都', 'prob': 1.0}],
  'predict': [{'name': '首都', 'prob': 0.999997615814209}]},
 {'text': '上海不是中国的首都',
  'text_span1': {'text': '上海', 'span': (0, 2), 'label': '城市', 'prob': 1.0},
  'text_span2': {'text': '中国', 'span': (4, 6), 'label': '国家', 'prob': 1.0},
  'label': [],
  'predict': []}]

In [17]:
true_labels = [e.label for e in to_pred]
eval_rs = eval_text_classify(true_labels, pred)
eval_rs

{'detail': {'中性': {'tp': 1,
   'fp': 0,
   'fn': 0,
   'precision': 1.0,
   'recall': 1.0,
   'f1': 1.0},
  '正向': {'tp': 1,
   'fp': 0,
   'fn': 0,
   'precision': 1.0,
   'recall': 1.0,
   'f1': 1.0},
  '负向': {'tp': 1,
   'fp': 0,
   'fn': 0,
   'precision': 1.0,
   'recall': 1.0,
   'f1': 1.0}},
 'micro': {'tp': 3,
  'fp': 0,
  'fn': 0,
  'precision': 1.0,
  'recall': 1.0,
  'f1': 1.0},
 'macro': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0}}

# save&load

In [18]:
model_path = os.path.join(config["common_config"]["experiment_dir"],
                          config["common_config"]["project_name"],
                          config["common_config"]["model_name"],"model")

save_args = dict(**config["common_config"]["save_args"])
del save_args["tf_serving_version"]

save_args
model_path


model.save(path=model_path, **save_args)




{'format': 'h5'}

'/nfs/pony/chenhao/experiment/config_ai_example/cls_token_classify_example/model'

2021-11-22 15:05:09 [INFO][core.py:79]:saving model to /nfs/pony/chenhao/experiment/config_ai_example/cls_token_classify_example/model
2021-11-22 15:05:09 [INFO][tf_core.py:55]:saving keras model to path:/nfs/pony/chenhao/experiment/config_ai_example/cls_token_classify_example/model/nn_model/nn_model.h5
2021-11-22 15:05:11 [INFO][core.py:144]:save model done


In [19]:
loaded_model = load_model(path=model_path)
loaded_model

2021-11-22 15:05:17 [INFO][core.py:90]:loading model from path:/nfs/pony/chenhao/experiment/config_ai_example/cls_token_classify_example/model
2021-11-22 15:05:17 [INFO][core.py:47]:init model with config:
2021-11-22 15:05:17 [INFO][core.py:124]:initializing tokenizer with config:
{
    "tokenizer_name": "bert_word_piece",
    "tokenizer_args": {
        "vocabs": "/nfs/pony/chenhao/pretrain/chinese_rbt4_L-4_H-768_A-12/vocab.txt"
    }
}
2021-11-22 15:05:17 [INFO][core.py:138]:tokenizer initialized with 21128 vocabs
2021-11-22 15:05:17 [INFO][tf_core.py:157]:loading keras model from path:/nfs/pony/chenhao/experiment/config_ai_example/cls_token_classify_example/model/nn_model/nn_model.h5 with format:h5
2021-11-22 15:05:20 [INFO][layer_utils.py:192]:Model: "nn_model"
2021-11-22 15:05:20 [INFO][layer_utils.py:193]:__________________________________________________________________________________________________
2021-11-22 15:05:20 [INFO][layer_utils.py:190]:Layer (type)                   

In [20]:
loaded_pred = loaded_model.predict(data=to_pred)
loaded_pred

100%|██████████| 3/3 [00:00<00:00, 11285.12it/s]
2021-11-22 15:05:23 [INFO][tf_core.py:204]:predicting with tf model...
2021-11-22 15:05:23 [INFO][decorators.py:28]:function:_model_predict cost:0.036 seconds
2021-11-22 15:05:23 [INFO][decorators.py:28]:function:_post_predict cost:0.001 seconds


[Label(name='正向', prob=0.9998229146003723),
 Label(name='负向', prob=0.9997743964195251),
 Label(name='中性', prob=0.9997645020484924)]